In [4]:
## Boiler plate code common to many notebooks.  See the TestFilesCommonCode.ipynb for details
from __future__ import print_function
%run TestFilesCommonCode.ipynb

SimpleITK Version: 0.9.1
Compiled: Sep 28 2015 10:07:41



In [28]:
import os
import glob
import sys

#\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/
#####################################################################################
#     Prepend the shell environment search paths
PROGRAM_PATHS = '/scratch/NAMICExternalProjects/release-20160523/bin:
/Shared/pinc/sharedopt/20141102/Slicer-4.4.0-macosx/Slicer.app/Contents/lib/Slicer-4.4/cli-modules'
#PROGRAM_PATHS = '/scratch/BS/release-BSR/bin'
PROGRAM_PATHS = PROGRAM_PATHS.split(':')
PROGRAM_PATHS.extend(os.environ['PATH'].split(':'))
os.environ['PATH'] = ':'.join(PROGRAM_PATHS)

CUSTOM_ENVIRONMENT=dict()

# Platform specific information
#     Prepend the python search paths
#PYTHON_AUX_PATHS = '/scratch/BS/BRAINSTools/AutoWorkup'
PYTHON_AUX_PATHS = '/scratch/SuperResolution/BRAINSSuperResolution/HCPWorkflows/'
PYTHON_AUX_PATHS = PYTHON_AUX_PATHS.split(':')
PYTHON_AUX_PATHS.extend(sys.path)
sys.path = PYTHON_AUX_PATHS

import SimpleITK as sitk
import nipype
from nipype.interfaces.base import CommandLine, CommandLineInputSpec, TraitedSpec, File, Directory
from nipype.interfaces.base import traits, isdefined, BaseInterface
from nipype.interfaces.utility import Merge, Split, Function, Rename, IdentityInterface
import nipype.interfaces.io as nio   # Data i/oS
import nipype.pipeline.engine as pe  # pypeline engine
from nipype.interfaces.freesurfer import ReconAll
from nipype.interfaces.ants import DenoiseImage
from nipype.interfaces.semtools import *

In [6]:
# # Normalize DWI scan components between zero and one
# def RescaleDWIComponents(inputDWI):
#     import os
#     import numpy as np
#     from ReadWriteNrrdDWI import ReadNAMICDWIFromNrrd, WriteNAMICDWIToNrrd
#     # utility function
#     def _3DRescale(arr):   
#         newMax = 4096
#         newMin = 0
#         oldMax = float(np.amax(arr))
#         oldMin = float(np.amin(arr))
#         f=(newMax-newMin)/(oldMax-oldMin)
#         rescaled_arr = (arr-oldMin)*f+newMin
#         return rescaled_arr
#     #
#     assert os.path.exists(inputDWI), "File not found: %s" % inputDWI
#     dwi_data,dwi_header,bvecs,bvals,gradient_index = ReadNAMICDWIFromNrrd(inputDWI)
#     numOfComponents = dwi_data.shape[gradient_index]
#     print(numOfComponents)
#     for idx in range(numOfComponents):
#         dwi_3d = dwi_data[:,:,:,idx]
#         dwi_3d_scaled = _3DRescale(dwi_3d)
#         dwi_data[:,:,:,idx] = dwi_3d_scaled
#     #outNormalizedDWIFilename = os.path.join(os.getcwd(), 'DWI_recaled.nrrd')
#     outdir='/Shared/johnsonhj/HDNI/20160804_HCP_Processing_Pipeline/Results/100307/newTractographyTest'
#     outNormalizedDWIFilename = os.path.join(outdir, 'DWI_SR_WTV_recaled.nrrd')
#     # write corrected nrrd file to disk
#     WriteNAMICDWIToNrrd(outNormalizedDWIFilename,dwi_data,bvecs,bvals,dwi_header)
#     assert os.path.isfile(outNormalizedDWIFilename), "Normalized DWI file is not found: %s" % outNormalizedDWIFilename
#     return outNormalizedDWIFilename

In [7]:
# Normalize DWI scan components between zero and one
def RescaleDWI(inputDWI):
    import os
    import numpy as np
    from ReadWriteNrrdDWI import ReadNAMICDWIFromNrrd, WriteNAMICDWIToNrrd
    #
    assert os.path.exists(inputDWI), "File not found: %s" % inputDWI
    dwi_data,dwi_header,bvecs,bvals,gradient_index = ReadNAMICDWIFromNrrd(inputDWI)
    #
    newMax = 4096
    newMin = 0
    oldMax = float(np.amax(dwi_data))
    oldMin = float(np.amin(dwi_data))
    f=(newMax-newMin)/(oldMax-oldMin)
    rescaled_dwi_data = (dwi_data-oldMin)*f+newMin
    rescaled_dwi_data = rescaled_dwi_data.astype(np.int16)
    #outNormalizedDWIFilename = os.path.join(os.getcwd(), 'DWI_rescaled.nrrd')
    outdir='/Shared/johnsonhj/HDNI/20160804_HCP_Processing_Pipeline/Results/100307/newTractographyTest'
    outNormalizedDWIFilename = os.path.join(outdir, 'DWI_SR_WTV_rescaled.nrrd')
    # write corrected nrrd file to disk
    WriteNAMICDWIToNrrd(outNormalizedDWIFilename,rescaled_dwi_data,bvecs,bvals,dwi_header)
    assert os.path.isfile(outNormalizedDWIFilename), "Normalized DWI file is not found: %s" % outNormalizedDWIFilename
    return outNormalizedDWIFilename

In [8]:
inputDWI='/Shared/johnsonhj/HDNI/20160804_HCP_Processing_Pipeline/Results/100307/Outputs/SuperResolution/DWI_SR_WTV.nrrd'
#inputDWI='/Shared/johnsonhj/HDNI/20160804_HCP_Processing_Pipeline/Results/100307/Outputs/SuperResolution/DWI_Baseline.nrrd'

In [9]:
#a = RescaleDWI(inputDWI)
#print(a)

/Shared/johnsonhj/HDNI/20160804_HCP_Processing_Pipeline/Results/100307/newTractographyTest/DWI_SR_WTV_rescaled.nrrd


In [22]:
from nipype.interfaces.slicer.diffusion import TractographyLabelMapSeeding

In [23]:
dwi_brain_mask='/Shared/johnsonhj/HDNI/20160804_HCP_Processing_Pipeline/Results/100307/Outputs/DWIBrainMask.nrrd'
input_dti='/Shared/johnsonhj/HDNI/20160804_HCP_Processing_Pipeline/Results/100307/newTractographyTest/DTI_WTV.nrrd'
fiber_output='/Shared/johnsonhj/HDNI/20160804_HCP_Processing_Pipeline/Results/100307/newTractographyTest/WTV_slicerTract_2.vtp'

In [31]:
tract = TractographyLabelMapSeeding()
tract.inputs.InputVolume = input_dti # input dti
tract.inputs.inputroi = dwi_brain_mask
tract.inputs.seedspacing = 1.25
tract.inputs.clthreshold = 0.1 # Minimum Linear Measure for the seeding to start
tract.inputs.minimumlength = 10
tract.inputs.maximumlength = 250
tract.inputs.stoppingmode = 'FractionalAnisotropy'
tract.inputs.stoppingvalue = 0.15
tract.inputs.stoppingcurvature = 0.7 # Tractography will stop if radius of curvature becomes smaller than this number units are degrees per mm
tract.inputs.integrationsteplength = 0.5 # Distance between points on the same fiber in mm
tract.inputs.label = 1
tract.inputs.OutputFibers = fiber_output
tract.run()

In [37]:
input_dwi='~/Desktop/DWI_Baseline.nrrd'
#input_dwi='~/Desktop/DWI_SR_WTV.nrrd'

sr_file_name = os.path.basename(input_dwi)
sr_file_name_base = os.path.splitext(sr_file_name)[0]
sr_name = sr_file_name_base + '_rescaled.nrrd'
print(sr_name)

DWI_Baseline_rescaled.nrrd
